In [22]:
!nvidia-smi

Sat Oct 15 23:04:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 511.23       Driver Version: 511.23       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:07:00.0  On |                  N/A |
| 35%   33C    P8    15W / 100W |   3448MiB /  4096MiB |     27%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [23]:
# Image2Image PipeLINE
import inspect
import warnings
from typing import List, Optional, Union

import torch
from torch import autocast
from tqdm.auto import tqdm

from diffusers import StableDiffusionImg2ImgPipeline

In [29]:
device = "cpu"
model_path = "CompVis/stable-diffusion-v1-4"

pipe = StableDiffusionImg2ImgPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", revision = "fp16", use_auth_token=True)

pipe = pipe.to(device)

{'trained_betas'} was not found in config. Values will be initialized to default values.


In [30]:
import requests
from io import BytesIO
from PIL import Image


In [31]:
def animalism(image_dir, prompt, strength, guidance_scale):
    init_img = Image.open(image_dir)
    generator = torch.Generator(device=device).manual_seed(1024)
#     with autocast("cuda"):
    image = pipe(prompt=prompt, init_image=init_img, strength=strength, guidance_scale=guidance_scale, generator=generator).images[0]

    return image

In [32]:
def makegrid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new("RGB", size=(cols*w, rows*h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    
    return grid

In [34]:
img_dir = "test.JPG"
prompt = "A cat looks like human, 4k, detailed, fantasy vivid colors"

created_img = animalism(img_dir, prompt, 0.7, 7.5)

created_img

  0%|          | 0/36 [00:00<?, ?it/s]

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 12 but got size 11 for tensor number 1 in the list.